In [1]:
import pandas as pd
import json
import requests
import config

In [2]:
url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'
key = '?registrationkey={}'.format(config.bls_key)

In [3]:
series_dict = {
    #1234567890123
    'CES7072250001': 'Restaurant Employees (thousands)',
    'CES7072250003': 'Restaurant Avg. Hourly', # of all Employees
    'CES7072250011': 'Restaurant Avg. Weekly', # earnings of all employees
    'CES7072250006': 'Restaurant PNS Employees (thousands)', # Production Non Supervisory
    'CES7072250008': 'Restaurant PNS Avg. Hourly', # Production Non Supervisory
    'CES7072250030': 'Restaurant PNS Avg. Weekly',
    'CES0500000001': 'Private Employees (thousands)', # Private Sector
    'CES0500000003': 'Private Avg. Hourly',
    'CES0500000011': 'Private Avg. Weekly',
    'CES0500000006': 'Private PNS Employees (thousands)', # Private PNS Sector
    'CES0500000008': 'Private PNS Avg. Hourly',
    'CES0500000030': 'Private PNS Avg. Weekly',
    'CES0000000001': 'Total Employees (thousands)', # Total NonFarm
}


dates = ('2013', '2023')

In [4]:
# Specify json as content type to return
headers = {'Content-type': 'application/json'}

# Submit the list of series as data
data = json.dumps({
    "seriesid": list(series_dict.keys()),
    "startyear": dates[0],
    "endyear": dates[1]})

# Post request for the data
p = requests.post(
    '{}{}'.format(url, key),
    headers=headers,
    data=data).json()['Results']['series']

In [14]:
# Date index from first series
date_list = [f"{i['year']}-{i['period'][1:]}-01" for i in p[0]['data']]

# Empty dataframe to fill with values
df = pd.DataFrame()

# Build a pandas series from the API results, p
for s in p:
    df[series_dict[s['seriesID']]] = pd.Series(
        index = pd.to_datetime(date_list),
        # Have to reverse copy, only first 114 entries, reverse copy again
        # because the mismatch in data lengths
        data = [i['value'] for i in s['data'][::-1][:114][::-1]],
        dtype = 'float64').iloc[::-1]

df

,Restaurant Employees (thousands),Restaurant Avg. Hourly,Restaurant Avg. Weekly,Restaurant PNS Employees (thousands),Restaurant PNS Avg. Hourly,Restaurant PNS Avg. Weekly,Private Employees (thousands),Private Avg. Hourly,Private Avg. Weekly,Private PNS Employees (thousands),Private PNS Avg. Hourly,Private PNS Avg. Weekly,Total Employees (thousands)
2013-01-01,9238.5,11.73,300.29,8241.3,10.47,256.52,113394.0,23.76,817.34,93659.0,19.94,669.98,135263.0
2013-02-01,9267.0,11.76,302.23,8266.0,10.50,257.25,113660.0,23.77,820.07,93870.0,19.99,675.66,135541.0
2013-03-01,9285.8,11.77,301.31,8289.2,10.51,257.50,113810.0,23.81,821.45,94001.0,20.01,676.34,135680.0
2013-04-01,9318.7,11.77,301.31,8310.5,10.52,257.74,114002.0,23.87,821.13,94155.0,20.04,675.35,135871.0
2013-05-01,9362.8,11.79,301.82,8360.8,10.55,258.48,114229.0,23.89,824.21,94302.0,20.05,675.69,136093.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-01,10478.8,17.77,451.36,9220.5,16.25,394.88,128295.0,31.60,1096.52,104516.0,26.95,921.69,150458.0
2022-03-01,10511.7,17.95,455.93,9236.8,16.38,399.67,128680.0,31.75,1098.55,104827.0,27.06,922.75,150856.0
2022-04-01,10563.3,18.01,455.65,9273.1,16.51,399.54,129048.0,31.86,1102.36,105141.0,27.18,926.84,151224.0
2022-05-01,10614.1,18.11,458.18,9295.5,16.54,400.27,129379.0,31.98,1106.51,105388.0,27.32,928.88,151610.0


In [15]:
df = df.reset_index()
df = df.rename(columns={'index': 'Date'})

In [16]:
df.to_csv('bls_722500.csv', index=False)